## Tarea 2
- Usar anomalies python package
- Implementar multiprocessing para calcular las soluciones

In [94]:
import numpy as np

import multiprocessing as multiprocessing

from anomalies import anomaly
from vector_like import vectorlike_generator

In [95]:
from operator import mul


def compute_anomaly_free_set(n):
    
    l, k = vectorlike_generator(n)
    
    set = anomaly.free(l, k)
    #print(set)
    gcd = anomaly.free.gcd
    
    if gcd==0:
        set_simplified = set
    else:
        set_simplified = anomaly.free.simplified
        
    results = {
        'l': l,
        'k': k,
        'set': set,
        'gcd': gcd,
        'set_simplified': set_simplified
    }
    
    return set_simplified#results


In [96]:
a = compute_anomaly_free_set(6)
a

array([  2,   3,   6, -13, -17,  19])

In [97]:
with multiprocessing.Pool(9) as p:
    info = p.map(compute_anomaly_free_set, [6]*1000)

/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/ana/anaconda3/envs/comp_avanz/lib/python3.9/site-packages/anomalies/a

In [98]:
sets = np.unique(np.vstack(info), axis=0)

In [99]:
sets = sets[np.all(sets!=0, axis=1)]
#sets

In [100]:
def repeated_values(array):
    
    pos = array[array>0]
    neg = array[array<0]

    #print(pos, neg, np.any(pos==np.abs(neg).any()))
    
    if np.all(pos==np.abs(neg)):
        return False

    elif np.any(pos==np.abs(neg).any()): 
        return False
    
    else:
        return True

In [101]:
#repeated_values(sets[1])

In [102]:
#np.any(np.array([2, 2, 3])==np.abs([-2, -2, -3]).any())
#np.array([2, 2, 3])==np.abs([-2, -2, -3]).any()

In [103]:
with multiprocessing.Pool(9) as p:
    remove = p.map(repeated_values, sets)

/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):
/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):
/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):
/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):
/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):
/tmp/ipykernel_13188/3721449450.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(pos==np.abs(neg)):


In [104]:
sets = sets[remove]
sets

array([[ -27,   29,   55,  -58,  -89,   90],
       [ -24,   26,   43,  -46,  -68,   69],
       [ -17,  -22,  -22,   72,   97, -108],
       [ -14,   21,   24,  -37,  -38,   44],
       [ -13,   15,   24,  -27,  -39,   40],
       [ -11,   16,   17,  -24,  -31,   33],
       [  -9,   11,   16,  -21,  -21,   24],
       [  -7,   10,   11,  -15,  -21,   22],
       [  -6,   29,   33,  -58, -149,  151],
       [  -3,   -5,   -7,   19,   21,  -25],
       [  -3,   -4,   -6,   16,   19,  -22],
       [  -2,    3,   10,  -13,  -13,   15],
       [  -2,    3,   51,  -53,  -73,   74],
       [  -1,   -5,   -7,   15,   21,  -23],
       [  -1,   -3,   -5,   11,   13,  -15],
       [  -1,   -2,   -5,   10,   11,  -13],
       [  -1,    2,    3,   -5,   -5,    6],
       [  -1,    3,    7,  -10,  -14,   15],
       [  -1,    3,   11,  -14,  -21,   22],
       [  -1,    4,    4,   -9,   -9,   11],
       [   2,    3,    6,  -13,  -17,   19],
       [   3,  -17,  -17,   33,   65,  -67],
       [  

In [105]:
"""

from concurrent.futures import thread
from threading import Thread

NUM_THREADS = 6

for i in range(NUM_THREADS):

    thread = Thread(target=compute_anomaly_free_set, args=(6,))
    thread.start()
    thread.join()

"""

'\n\nfrom concurrent.futures import thread\nfrom threading import Thread\n\nNUM_THREADS = 6\n\nfor i in range(NUM_THREADS):\n\n    thread = Thread(target=compute_anomaly_free_set, args=(6,))\n    thread.start()\n    thread.join()\n\n'